In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import (MultipleLocator,FormatStrFormatter, AutoMinorLocator)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
# load the Per Game Stats of NBA Players
Playerstats = pd.read_csv('source_data/NBA_Player_Stats_20-21.csv',  encoding="utf-8")
Playerstats['Player'] = Playerstats['Player'].apply(lambda x : x.split("\\")[0])
Tm = Playerstats.pop('Tm')
Playerstats.insert(2, 'Tm', Tm)
Playerstats = Playerstats.drop(['Rk'], axis = 1)
Playerstats

In [ ]:
# load the Salary of NBA Players
PlayerSalary =  pd.read_csv('source_data/NBA_Player_Salary_20-21.txt', encoding="utf-8")
PlayerSalary['Player'] = PlayerSalary['Player'].apply(lambda x : x.split("\\")[0])
PlayerSalary = PlayerSalary[PlayerSalary['Salary'].notnull()]
PlayerSalary = PlayerSalary.drop(['Rk'], axis = 1)
PlayerSalary


In [ ]:
# load the Advanced Stats of NBA Players
PlayerAdvanced =  pd.read_csv('source_data/NBA_Player_Advanced_20-21.csv', encoding="utf-8")
PlayerAdvanced['Player'] = PlayerAdvanced['Player'].apply(lambda x : x.split("\\")[0])
Tm = PlayerAdvanced.pop('Tm')
PlayerAdvanced.insert(2, 'Tm', Tm)
PlayerAdvanced = PlayerAdvanced.drop(['Rk', 'MP', 'G', 'Pos', 'Age', 'Unnamed: 19'], axis = 1)
PlayerAdvanced

In [ ]:
# join the three datasets above with the order as Salary dataset + Stats dataset + Advanced dataset
Player_Salary_Stats = PlayerSalary.join(Playerstats.set_index(['Player', 'Tm']), on=['Player', 'Tm'])
Player_Salary_Stats
Player_Salary_Stats_Advanced = Player_Salary_Stats.join(PlayerAdvanced.set_index(['Player', 'Tm']), on=['Player', 'Tm'])
# make the data in Salary column from string to int
Player_Salary_Stats_Advanced['Salary'] = Player_Salary_Stats_Advanced['Salary'].str.replace(r"$", "").astype(int)
Player_Salary_Stats_Advanced = Player_Salary_Stats_Advanced[Player_Salary_Stats_Advanced['PER'].notnull()]
Player_Salary_Stats_Advanced.reset_index(drop=True)
Player_Salary_Stats_Advanced.to_csv('NBA_Player_data_comprehensive.csv', index=False)

In [19]:
# adding Team abbreviations to Team_records.csv

team_abb = {'Atlanta Hawks': 'ATL', 'Brooklyn Nets': 'BRK', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHA',
           'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Dallas Mavericks': 'DAL',
           'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW',
           'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 'Los Angeles Clippers': 'LAC',
           'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM', 'Miami Heat': 'MIA',
           'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 'New Orleans Pelicans': 'NOP',
           'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL',
           'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHX', 'Portland Trail Blazers': 'POR',
           'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Toronto Raptors': 'TOR',
           'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}

record = pd.read_csv('source_data/Team_records.csv', encoding = 'utf-8')
record = record.rename(columns = {'Unnamed: 0': 'Rk', 'Unnamed: 1': 'Team', 'Unnamed: 2': 'Overall'})
record = record.drop([0,1])
record = record.reset_index().drop('index', axis = 1)

record['Team_abb'] = ''

for i in range(record.shape[0]):
    record['Team_abb'][i] = team_abb[record['Team'][i]]

# rearrange the columns
record = record[['Rk','Team','Team_abb','Overall']]

# fix the Timberwolves record
record.iloc[29,3] = '11-36'

# add the win percentage column
record['Win%'] = ''
for i in range(record.shape[0]):
    record_list = record.iloc[i,3].split('-')
    record.iloc[i,4] = int(record_list[0]) / (int(record_list[0]) + int(record_list[1]))

record.to_csv('team_record_comprehensive.csv', index=False)